In [1]:
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import SparkSession

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
import pyspark.sql.functions as F

date = '2022-06-21'
conf = SparkConf().setAppName(f"EventsPartitioningJob-{date}")
sc = SparkContext(conf=conf)
sql = SQLContext(sc)

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/lib/spark/jars/slf4j-log4j12-1.7.30.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/lib/hadoop/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2023-08-14 19:39:22,616 WARN util.Utils: Your hostname, fhm3dgacl4cedf240j72 resolves to a loopback address: 127.0.1.1; using 172.16.0.29 instead (on interface eth0)
2023-08-14 19:39:22,617 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-08-14 19:39:24,528 WARN util.Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update 

In [5]:
print(sql)

In [6]:
base_input_path = '/user/master/data/events'
events = sql.read.json(f"{base_input_path}/date={date}")

In [8]:
def partition_writer(events):
    return events \
        .write \
        .mode('overwrite') \
        .partitionBy('event_type') \
        .format('parquet')

In [9]:
base_output_path = '/user/intgemidde/data/events'
writer = partition_writer(events)
writer.save(f'{base_output_path}/date={date}')